In [3]:
pip install nltk scikit-learn streamlit


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\harsh\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [9]:
import random
import nltk
import numpy as np
import string
import sqlite3
import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

# Expanded intents dataset
intents = [
    {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "How are you", "What's up", "Hi there", "Hello there"],
        "responses": ["Hi there!", "Hello!", "Hey!", "I'm fine, thank you!", "Nothing much."]
    },
    {
        "tag": "goodbye",
        "patterns": ["Bye", "See you later", "Goodbye", "Take care", "Farewell", "Catch you later"],
        "responses": ["Goodbye!", "See you later!", "Take care!", "Farewell!"]
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it", "Much appreciated"],
        "responses": ["You're welcome!", "No problem!", "Glad I could help!", "Anytime!"]
    },
    {
        "tag": "developer",
        "patterns": ["Who is your developer", "Who created you", "Who built you"],
        "responses": ["My developer is Suraj Sharma.", "I was created by Suraj Sharma."]
    }
]

# Build training data
all_patterns = []
all_labels = []
responses = {}

for intent in intents:
    for pattern in intent['patterns']:
        all_patterns.append(pattern)
        all_labels.append(intent['tag'])
    responses[intent['tag']] = intent['responses']

def clean_up_sentence(sentence):
    """
    Removes punctuation, tokenizes, and lemmatizes the sentence.
    """
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    sentence_words = nltk.word_tokenize(sentence.lower())
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return ' '.join(sentence_words)  # Convert list back to string

# Ensure all patterns are strings
all_patterns = [str(pattern) for pattern in all_patterns]

vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize, token_pattern=None)  # Fix warning
X = vectorizer.fit_transform(all_patterns)

classifier = LogisticRegression(max_iter=200)
classifier.fit(X, all_labels)

def get_response(user_input, threshold=0.1):
    """
    Predicts the intent of the user_input and returns an appropriate response.
    """
    input_vector = vectorizer.transform([user_input])
    probabilities = classifier.predict_proba(input_vector)[0]
    max_prob = max(probabilities)

    if max_prob < threshold:
        return "I'm sorry, I didn't quite understand that. Could you please rephrase?"
    
    predicted_tag = classifier.predict(input_vector)[0]
    return random.choice(responses[predicted_tag])

# Database setup using SQLite
def create_db():
    conn = sqlite3.connect("chatbot.db")
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS chat_logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            user_message TEXT,
            bot_response TEXT
        )
    ''')
    conn.commit()
    conn.close()

def log_conversation(user_message, bot_response):
    """
    Inserts a record of the conversation into the chat_logs table.
    """
    conn = sqlite3.connect("chatbot.db")
    cursor = conn.cursor()
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cursor.execute("INSERT INTO chat_logs (timestamp, user_message, bot_response) VALUES (?, ?, ?)",
                   (timestamp, user_message, bot_response))
    conn.commit()
    conn.close()

def chat():
    """
    Starts an interactive chat with the user.
    """
    create_db()  # Ensure database table is created
    print("Chatbot: Hi, I'm your friendly chatbot. How can I help you today? (type 'quit' to exit)")

    while True:
        user_input = input("You: ").strip().lower()
        if user_input in ["quit", "exit"]:
            print("Chatbot: Goodbye!")
            break
        
        response = get_response(user_input)
        print(f"Chatbot: {response}")
        log_conversation(user_input, response)

if __name__ == "__main__":
    chat()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\harsh/nltk_data'
    - 'C:\\Users\\harsh\\AppData\\Local\\Programs\\Python\\Python313\\nltk_data'
    - 'C:\\Users\\harsh\\AppData\\Local\\Programs\\Python\\Python313\\share\\nltk_data'
    - 'C:\\Users\\harsh\\AppData\\Local\\Programs\\Python\\Python313\\lib\\nltk_data'
    - 'C:\\Users\\harsh\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
